<span style="float:left;">Licence CC BY-SA</span><span style="float:right;">Fabrice Zaoui - Cedric Goeury&nbsp;</span><br/>
___

This tutorial is intended for people who want an example showing how to optimize a Telemac 2D case with a genetic algorithm based on Genop Python module.

# Import Optimizer from TelApy

In [1]:
from telapy.tools import genop

The module 'genop' is located in the '$HOMETEL/scripts/python27/TelApy/tools' directory. If an error occurs while attempting to import, check the value of the environment variable PYTHONPATH.


'genop' uses a meta-heuristic (the Genetic Algorithm) for a mono-objective optimization.

In [2]:
help(genop)

Help on package telapy.tools.genop in telapy.tools:

NAME
    telapy.tools.genop - GENOP : a Python (v3) Genetic Optimizer

DESCRIPTION
    Adapted from the Scilab module 'ga_optim'
    
    Auteur : Fabrice Zaoui (EDF R&D LNHE)
    
    Copyright EDF 2016-2018

PACKAGE CONTENTS
    costfunction
    crossover
    genop
    genpop
    mutation
    selection
    showresults
    validate

CLASSES
    builtins.object
        telapy.tools.genop.genop.Genop
    
    class Genop(builtins.object)
     |  The base class for the Genetic Opimization
     |  
     |  Methods defined here:
     |  
     |  __init__(self, popsize=25, pbcross=0.7, pbmut=0.1)
     |      Initialize some algorithmic parameters to default values
     |      :return: a new object from Genop
     |  
     |  initialize(self, func, nvar, bounds)
     |      Description of the minimization problem
     |      :param 'f': the name of the python function where the cost function
     |          is implemented (type: str)
     

# The Telemac2d test case

A problem is created with the 'estimation' example located in '$HOMETEL/examples/telemac2d/estimation'.

The test case 'estimation' is dedicated to the automatic calibration problem. This Telemac-2D model is composed of 551 triangular mesh elements. On the upstream of the model, an imposed flow type boundary condition is used. On downstream, a water depth is applied.

The script 'study_t2d.py' has been written to manage this case with the Telemac API (initialization of the case, run of the case, get some important values on the physical variables).

In [3]:
from telapy.tools.study_t2d import StudyTelemac2D
import numpy as np
import os

# Goal

The calibration aims at minimizing the error between observations and Telemac computations on the water depths. Six observations are available on the middle of the domain. A norm is used to evaluate the error as follows:

In [4]:
def estimation(CHESTR):
    YObs = [5.000000000000000000e-01, 7.517873224063527093e-01, \
            7.517873219825667030e-01, 7.517873219442824384e-01, \
            7.517873221409325790e-01, 7.517873218929342904e-01]
    YObs = np.asmatrix(np.ravel(YObs)).T # six observations
    Fx = study.h_x(CHESTR[0]) # Telemac computation with new friction coefficient
    Res = np.linalg.norm(YObs -Fx) # The norm evaluation
    return np.array(Res)

# Reading the case

In [5]:
# Changing of directory : 'examples' directory of the Telemac sources
HOMETEL = os.environ.get('HOMETEL')
os.chdir(HOMETEL + '/examples/telemac2d/estimation')

In [6]:
#  Telemac 2d files
studyFiles={'t2d.f'   :'user_fortran',\
            't2d.cas' :'t2d_estimation_basic.cas',\
            'f2d.slf' :'f2d_estimation.slf',\
            't2d.geo' :'geo_estimation.slf' }

In [7]:
# Observation times
obs_time = [0.0, 2000.0, 4000.0, 6000.0, 8000.0, 10000.0]
# A polygon is defined to get the observation node
poly = [(246.114, 57.3554), (261.13, 57.0189), (261.802, 45.018), (245.666, 45.3545)]

# Initialization

In [8]:
# Class Instantiation 
study = StudyTelemac2D(studyFiles, obs_time, poly)

# Run the optimization

WARNINGS: The optimization will take time because the required number of Telemac runs is significant.

In [ ]:
# pointer to the function
fcost = estimation
# number of variables
nvar = 1
# lower and upper bounds for variable (optimizing in the range [20., 60.])
bounds = np.zeros((nvar, 2))
bounds[0, 0] = 20.
bounds[0, 1] = 60.

# instantiation of a Genop problem with default values for parameters
mypb = genop.Genop()
# initialization of the problem for Genop
error = mypb.initialize(fcost, nvar, bounds)
if error:
    raise Exception('Unable to initialize Genop')

# run optimization in parallel mode
# ...comments: change parameter nproc depending on your machine 
fcost, xopt = mypb.optimize(nproc=3)

In [ ]:
# Print all the optimal solutions found for the friction coefficient
print(xopt)

In [ ]:
# Print the corresponding errors (2-norm)
print(fcost)